<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/brace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:

#Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%cd /content/drive/MyDrive/Github/capstone/brace

/content/drive/MyDrive/Github/capstone/brace


In [18]:
%ls

annotations/  dataset_pytorch.py  __pycache__/  utils/
dataset/      figures/            README.md     videos_info.csv


In [20]:
import sys
sys.path.append('/content/drive/MyDrive/Github/capstone/brace')

In [23]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import dataset_pytorch
from pathlib import Path

In [42]:
sequences_path = Path('/content/drive/MyDrive/Github/capstone/brace/dataset')  # Adjust this path to where your dataset is located
annotations_path = Path('/content/drive/MyDrive/Github/capstone/brace/annotations')
df_ = pd.read_csv(Path('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences.csv'))

In [43]:
# train_df = pd.read_csv('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences_train.csv')
# test_df = pd.read_csv('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences_test.csv')

In [64]:
segments= pd.read_csv('/content/drive/MyDrive/Github/capstone/brace/annotations/segments.csv')
segments

,video_id,seq_idx,start_frame,end_frame,dance_type,dance_type_id,dancer,dancer_id,year,uid
0,3rIk56dcBTM,0,851,1221,toprock,0,el niño,10,2011,3rIk56dcBTM.851.1221
1,3rIk56dcBTM,0,1234,1330,powermove,1,el niño,10,2011,3rIk56dcBTM.1234.1330
2,3rIk56dcBTM,0,1348,1555,footwork,2,el niño,10,2011,3rIk56dcBTM.1348.1555
3,3rIk56dcBTM,0,1571,1645,powermove,1,el niño,10,2011,3rIk56dcBTM.1571.1645
4,3rIk56dcBTM,1,1751,2085,toprock,0,roxrite,47,2011,3rIk56dcBTM.1751.2085
...,...,...,...,...,...,...,...,...,...,...
1347,zvWiTWT3T2M,5,4198,4270,toprock,0,alkolil,0,2020,zvWiTWT3T2M.4198.4270
1348,zvWiTWT3T2M,5,4281,4391,powermove,1,alkolil,0,2020,zvWiTWT3T2M.4281.4391
1349,zvWiTWT3T2M,5,4407,4563,footwork,2,alkolil,0,2020,zvWiTWT3T2M.4407.4563
1350,zvWiTWT3T2M,5,4576,4654,powermove,1,alkolil,0,2020,zvWiTWT3T2M.4576.4654


In [63]:
train_df

,video_id,seq_idx,start_frame,end_frame,dancer,dancer_id,year,uid
1,3rIk56dcBTM,1,1751,2915,roxrite,47,2011,3rIk56dcBTM.1
2,3rIk56dcBTM,2,3012,3764,el niño,10,2011,3rIk56dcBTM.2
3,3rIk56dcBTM,3,3851,4496,roxrite,47,2011,3rIk56dcBTM.3
5,A_KxOaDHi6o,1,1473,2410,roxrite,47,2011,A_KxOaDHi6o.1
6,A_KxOaDHi6o,2,2524,3334,soso,51,2011,A_KxOaDHi6o.2
...,...,...,...,...,...,...,...,...
454,x3uelTisF6g,1,1330,2047,kid karam,21,2020,x3uelTisF6g.1
457,x3uelTisF6g,4,4282,5135,shigekix,49,2020,x3uelTisF6g.4
458,x3uelTisF6g,5,5241,6089,kid karam,21,2020,x3uelTisF6g.5
459,zvWiTWT3T2M,0,114,967,lil zoo,27,2020,zvWiTWT3T2M.0


In [51]:
train_df = pd.read_csv(annotations_path/'sequences_train.csv')
train_df = df_[df_.uid.isin(train_df.uid)]
test_df = pd.read_csv(annotations_path/'sequences_test.csv')
test_df = df_[df_.uid.isin(test_df.uid)]

In [66]:
train_df

,video_id,seq_idx,start_frame,end_frame,dancer,dancer_id,year,uid
1,3rIk56dcBTM,1,1751,2915,roxrite,47,2011,3rIk56dcBTM.1
2,3rIk56dcBTM,2,3012,3764,el niño,10,2011,3rIk56dcBTM.2
3,3rIk56dcBTM,3,3851,4496,roxrite,47,2011,3rIk56dcBTM.3
5,A_KxOaDHi6o,1,1473,2410,roxrite,47,2011,A_KxOaDHi6o.1
6,A_KxOaDHi6o,2,2524,3334,soso,51,2011,A_KxOaDHi6o.2
...,...,...,...,...,...,...,...,...
454,x3uelTisF6g,1,1330,2047,kid karam,21,2020,x3uelTisF6g.1
457,x3uelTisF6g,4,4282,5135,shigekix,49,2020,x3uelTisF6g.4
458,x3uelTisF6g,5,5241,6089,kid karam,21,2020,x3uelTisF6g.5
459,zvWiTWT3T2M,0,114,967,lil zoo,27,2020,zvWiTWT3T2M.0


In [65]:
y_train = segments[segments.uid.isin(train_df.uid)]
y_train

,video_id,seq_idx,start_frame,end_frame,dance_type,dance_type_id,dancer,dancer_id,year,uid


In [52]:
# Initialize the datasets
brace_train = dataset_pytorch.BraceDataset(sequences_path, train_df)
brace_test = dataset_pytorch.BraceDataset(sequences_path, test_df)

Loading BRACE: 100%|██████████| 319/319 [04:07<00:00,  1.29it/s]


Going to sample 900 frames from each sequence. Average sequence length is 730.7711598746082


Loading BRACE: 100%|██████████| 146/146 [01:53<00:00,  1.29it/s]

Going to sample 900 frames from each sequence. Average sequence length is 690.9794520547945


In [61]:
for i in range(1):  # Adjust the range as needed
    sample = brace_train[i]
    print(sample)

(array([[[1.59639289e-02, 3.15346935e-02],
        [1.64896881e-02, 7.89936409e-03],
        [0.00000000e+00, 7.83592621e-03],
        ...,
        [4.01691696e-01, 7.85855145e-01],
        [1.00000000e+00, 9.07045814e-01],
        [3.83193401e-01, 1.00000000e+00]],

       [[0.00000000e+00, 3.38604408e-02],
        [1.89239301e-02, 8.46178845e-03],
        [5.38366605e-04, 1.07911813e-02],
        ...,
        [6.40482841e-01, 7.73193886e-01],
        [1.00000000e+00, 8.91722771e-01],
        [6.33910754e-01, 1.00000000e+00]],

       [[0.00000000e+00, 3.74846454e-02],
        [2.92187883e-02, 1.17085473e-02],
        [2.38255090e-03, 1.34343050e-02],
        ...,
        [8.44325446e-01, 7.56283738e-01],
        [9.05700638e-01, 9.11636007e-01],
        [9.98535006e-01, 1.00000000e+00]],

       ...,

       [[6.52031936e-01, 2.34444789e-02],
        [6.55712239e-01, 0.00000000e+00],
        [5.67201410e-01, 1.03653876e-02],
        ...,
        [3.48476937e-01, 7.41583570e-01],
    